In [77]:
import os 
import csv

import influxdb_client
from influxdb_client.client.write_api import ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError


INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=hola"

client = None
try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    health = client.health()

    if health.status == "pass":
        print("Conexión exitosa")
    else:
        print("Conexión fallida")

except(InfluxDBError,NewConnectionError) as e:
    print("Error")
    print(e)

Verificando estado de salud de InfluxDB en http://influxdb2:8086...
Conexión exitosa


In [81]:
query_api = client.query_api()

query = """
from(bucket: "crypto_raw")
  |> range(start: 2020-12-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "BTC")
  |> filter(fn: (r) => r["_field"] == "Close")
"""

tables = query_api.query(query)

for table in tables:
    for record in table.records:
        print(f"Tiempo: {record.get_time()} | Simbolo: {record['symbol']} | Precio Cierre: {record['_value']}")

Tiempo: 2020-12-01 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18802.99829969
Tiempo: 2020-12-02 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 19201.09115697
Tiempo: 2020-12-03 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 19445.39847988
Tiempo: 2020-12-04 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18699.76561337
Tiempo: 2020-12-05 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 19154.231131
Tiempo: 2020-12-06 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 19345.12095871
Tiempo: 2020-12-07 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 19191.63128698
Tiempo: 2020-12-08 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18321.14491611
Tiempo: 2020-12-09 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18553.91537685
Tiempo: 2020-12-10 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18264.99210672
Tiempo: 2020-12-11 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18058.90334725
Tiempo: 2020-12-12 23:59:59+00:00 | Simbolo: BTC | Precio Cierre: 18803.65687045
Tiempo: 2020-12-13 23:59:59+00

In [ ]:
query = """
from(bucket: "crypto_raw")
  |> range(start: 2021-01-01T00:00:00Z, stop: 2021-07-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "ETH")
  |> filter(fn: (r) => r["_field"] == "volume")
  |> sum()
"""

results = query_api.query(query)

for table in results:
    for record in table.records:
        print(f"{record['symbol']} --> Volumen Total: {record.get_value()}")

ETH --> Volumen Total: 6239381609386.877


In [ ]:
query = """
from(bucket: "crypto_raw")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "BTC")
  |> filter(fn: (r) => r["_field"] == "Close")
  |> aggregateWindow(every: 1mo, fn: mean, createEmpty: false)
"""

results = query_api.query(query)

print("--- Promedio Mensual BTC ---")
for table in results:
    for record in table.records:
        fecha = record.get_time().strftime('%Y-%m')
        promedio = record.get_value()
        print(f"Mes: {fecha} | Precio Promedio: {promedio:.2f}")

--- Promedio Mensual BTC ---
Mes: 2013-05 | Precio Promedio: 141.77
Mes: 2013-06 | Precio Promedio: 119.99
Mes: 2013-07 | Precio Promedio: 107.76
Mes: 2013-08 | Precio Promedio: 90.51
Mes: 2013-09 | Precio Promedio: 113.91
Mes: 2013-10 | Precio Promedio: 130.06
Mes: 2013-11 | Precio Promedio: 158.31
Mes: 2013-12 | Precio Promedio: 550.42
Mes: 2014-01 | Precio Promedio: 800.78
Mes: 2014-02 | Precio Promedio: 844.17
Mes: 2014-03 | Precio Promedio: 661.62
Mes: 2014-04 | Precio Promedio: 592.20
Mes: 2014-05 | Precio Promedio: 461.36
Mes: 2014-06 | Precio Promedio: 486.65
Mes: 2014-07 | Precio Promedio: 615.92
Mes: 2014-08 | Precio Promedio: 618.03
Mes: 2014-09 | Precio Promedio: 536.09
Mes: 2014-10 | Precio Promedio: 445.19
Mes: 2014-11 | Precio Promedio: 364.15
Mes: 2014-12 | Precio Promedio: 366.10
Mes: 2015-01 | Precio Promedio: 341.27
Mes: 2015-02 | Precio Promedio: 248.78
Mes: 2015-03 | Precio Promedio: 234.15
Mes: 2015-04 | Precio Promedio: 269.04
Mes: 2015-05 | Precio Promedio: 235.

In [ ]:
query = """
data = from(bucket: "crypto_raw")
  |> range(start: 2019-01-01T00:00:00Z, stop: 2020-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "XLM")
  |> filter(fn: (r) => r["_field"] == "Close")

data
  |> aggregateWindow(every: 1w, fn: max, createEmpty: false)
  |> yield(name: "maximo")

data
  |> aggregateWindow(every: 1w, fn: min, createEmpty: false)
  |> yield(name: "minimo")
"""

results = query_api.query(query)

for table in results:
    if len(table.records) > 0:
        tipo_agregacion = table.records[0]["result"]
        print(f"Resultados de: {tipo_agregacion}")
        
        for record in table.records:

            fecha = record.get_time().strftime('%Y-%m-%d')
            valor = record.get_value()
            print(f"Fecha: {fecha} | Valor: {valor}")

Resultados de: minimo
Fecha: 2019-01-03 | Valor: 0.11593003965
Fecha: 2019-01-10 | Valor: 0.113824645405
Fecha: 2019-01-17 | Valor: 0.103608750666
Fecha: 2019-01-24 | Valor: 0.102576372742
Fecha: 2019-01-31 | Valor: 0.0839227197999
Fecha: 2019-02-07 | Valor: 0.0743873347098
Fecha: 2019-02-14 | Valor: 0.0746880850898
Fecha: 2019-02-21 | Valor: 0.076635145311
Fecha: 2019-02-28 | Valor: 0.0842337176392
Fecha: 2019-03-07 | Valor: 0.0835385127758
Fecha: 2019-03-14 | Valor: 0.0855406472047
Fecha: 2019-03-21 | Valor: 0.107271925309
Fecha: 2019-03-28 | Valor: 0.102469191942
Fecha: 2019-04-04 | Valor: 0.107359387845
Fecha: 2019-04-11 | Valor: 0.11878829353
Fecha: 2019-04-18 | Valor: 0.113628616719
Fecha: 2019-04-25 | Valor: 0.103259281028
Fecha: 2019-05-02 | Valor: 0.0964745129392
Fecha: 2019-05-09 | Valor: 0.0936971600069
Fecha: 2019-05-16 | Valor: 0.0902320403193
Fecha: 2019-05-23 | Valor: 0.124257250797
Fecha: 2019-05-30 | Valor: 0.125166646196
Fecha: 2019-06-06 | Valor: 0.121283780398
Fecha

In [ ]:
query = """
from(bucket: "crypto_raw")
  |> range(start: 2016-01-01T00:00:00Z, stop: 2019-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "USDT")
  |> filter(fn: (r) => r["_field"] == "Close")
  |> aggregateWindow(every: 1d, fn: last)
  |> difference()
  |> map(fn: (r) => ({ r with porcentaje: r._value * 100.0 })) 
"""

results = query_api.query(query)

print(f"{'Fecha':<12} | {'Cambio $':<12} | {'Cambio %':<10}")

for table in results:
    for record in table.records:
        fecha = record.get_time().strftime('%Y-%m-%d')
        diff_usd = record.get_value()
        print(f"{fecha:<12} | {diff_usd:<12.6f} | {record['porcentaje']:.4f}%")

Fecha        | Cambio $     | Cambio %  
2016-01-03   | 0.000000     | 0.0000%
2016-01-04   | 0.000000     | 0.0000%
2016-01-05   | 0.000000     | 0.0000%
2016-01-06   | 0.000250     | 0.0250%
2016-01-07   | -0.000250    | -0.0250%
2016-01-08   | 0.000000     | 0.0000%
2016-01-09   | 0.000000     | 0.0000%
2016-01-10   | 0.000000     | 0.0000%
2016-01-11   | 0.000000     | 0.0000%
2016-01-12   | 0.000000     | 0.0000%
2016-01-13   | 0.000000     | 0.0000%
2016-01-14   | 0.000450     | 0.0450%
2016-01-15   | -0.000450    | -0.0450%
2016-01-16   | 0.000000     | 0.0000%
2016-01-17   | 0.000410     | 0.0410%
2016-01-18   | -0.000410    | -0.0410%
2016-01-19   | 0.000000     | 0.0000%
2016-01-20   | 0.000000     | 0.0000%
2016-01-21   | 0.000000     | 0.0000%
2016-01-22   | 0.000000     | 0.0000%
2016-01-23   | 0.000070     | 0.0070%
2016-01-24   | 0.000080     | 0.0080%
2016-01-25   | -0.000140    | -0.0140%
2016-01-26   | 0.000000     | 0.0000%
2016-01-27   | -0.000010    | -0.0010%
2016

In [ ]:
query = """
btc_data = from(bucket: "crypto_raw")
  |> range(start: 2020-01-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "BTC")
  |> filter(fn: (r) => r["_field"] == "Close")

eth_data = from(bucket: "crypto_raw")
  |> range(start: 2020-01-01T00:00:00Z, stop: 2021-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Criptos")
  |> filter(fn: (r) => r["symbol"] == "ETH")
  |> filter(fn: (r) => r["_field"] == "Close")

join(tables: {btc: btc_data, eth: eth_data}, on: ["_time"])
  |> map(fn: (r) => ({
      _time: r._time,
      btc_price: r._value_btc,
      eth_price: r._value_eth,
      ratio: r._value_btc / r._value_eth
    }))
"""

results = query_api.query(query)

print(f"{'Fecha':<12} | {'BTC':<10} | {'ETH':<10} | {'Ratio':<10}")


for table in results:
    for record in table.records:
        fecha = record.get_time().strftime('%Y-%m-%d')
        btc = record["btc_price"]
        eth = record["eth_price"]
        ratio = record["ratio"]
        print(f"{fecha:<12} | {btc:<10.2f} | {eth:<10.2f} | {ratio:<10.4f}")

client.close()

Fecha        | BTC        | ETH        | Ratio     
2020-01-01   | 7200.17    | 130.80     | 55.0464   
2020-01-02   | 6985.47    | 127.41     | 54.8266   
2020-01-03   | 7344.88    | 134.17     | 54.7424   
2020-01-04   | 7410.66    | 135.07     | 54.8656   
2020-01-05   | 7411.32    | 136.28     | 54.3843   
2020-01-06   | 7769.22    | 144.30     | 53.8392   
2020-01-07   | 8163.69    | 143.54     | 56.8724   
2020-01-08   | 8079.86    | 141.26     | 57.1993   
2020-01-09   | 7879.07    | 138.98     | 56.6925   
2020-01-10   | 8166.55    | 143.96     | 56.7264   
2020-01-11   | 8037.54    | 142.93     | 56.2352   
2020-01-12   | 8192.49    | 145.87     | 56.1615   
2020-01-13   | 8144.19    | 144.23     | 56.4681   
2020-01-14   | 8827.76    | 165.96     | 53.1936   
2020-01-15   | 8807.01    | 166.23     | 52.9807   
2020-01-16   | 8723.79    | 164.39     | 53.0673   
2020-01-17   | 8929.04    | 170.78     | 52.2839   
2020-01-18   | 8942.81    | 175.37     | 50.9952   
2020-01-19  